#DV360 Bulk Targeting Editor
Allows bulk targeting DV360 through Sheets and BigQuery.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Set Configuration

This code is required to initialize the project. Fill in required fields and press play.

1. If the recipe uses a Google Cloud Project:
  - Set the configuration **project** value to the project identifier from [these instructions](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md).

1. If the recipe has **auth** set to **user**:
  - If you have user credentials:
    - Set the configuration **user** value to your user credentials JSON.
  - If you DO NOT have user credentials:
    - Set the configuration **client** value to [downloaded client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md).

1. If the recipe has **auth** set to **service**:
  - Set the configuration **service** value to [downloaded service credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_service.md).



In [ ]:
from starthinker.util.configuration import Configuration


CONFIG = Configuration(
  project="",
  client={},
  service={},
  user="/content/user.json",
  verbose=True
)



#3. Enter DV360 Bulk Targeting Editor Recipe Parameters
 1. Select **Load**, click **Save + Run**, a sheet called ** DV Targeter** will be created.
 1. In the **Partners** sheet tab, fill in **Filter** column then select **Load**, click **Save + Run**.
 1. In the **Advertisers** sheet tab, fill in **Filter** column. then select **Load**, click **Save + Run**.
 1. Check the First And Third Party option to load audiences, which may be slow.  If not loaded, user will enter audience ids into the sheet manually.
 1. On the **Line Items** sheet tab, the **Filter** is used only to limit drop down choices in the rest of the tool.
 1. Optionally edit or filter the **Targeting Options** or **Inventory Sources** sheets to limit choices.
 1. Make targeting updates, fill in changes on all tabs with colored fields (RED FIELDS ARE NOT IMPLEMENTED, IGNORE).
 1. Select **Preview**, click **Save + Run** then check the **Preview** tabs.
 1. Select **Update**, click **Save + Run** then check the **Success** and **Error** tabs.
 1. Load and Update can be run multiple times.
 1. If an update fails, all parts of the update failed, break it up into multiple updates.
 1. To refresh the Partner, Advertiser, or Line Item list, remove the filters and run load.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_dv':'user',  # Credentials used for dv.
  'auth_sheet':'user',  # Credentials used for sheet.
  'auth_bigquery':'service',  # Credentials used for bigquery.
  'recipe_name':'',  # Name of Google Sheet to create.
  'recipe_slug':'',  # Name of Google BigQuery dataset to create.
  'command':'Load',  # Action to take.
  'first_and_third':False,  # Load first and third party data (may be slow). If not selected, enter audience identifiers into sheet manually.
}

print("Parameters Set To: %s" % FIELDS)


#4. Execute DV360 Bulk Targeting Editor
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

TASKS = [
  {
    'dataset':{
      '__comment__':'Ensure dataset exists.',
      'auth':{'field':{'name':'auth_bigquery','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','order':2,'default':'','description':'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'drive':{
      '__comment__':'Copy the default template to sheet with the recipe name',
      'auth':{'field':{'name':'auth_sheet','kind':'authentication','order':1,'default':'user','description':'Credentials used for reading data.'}},
      'copy':{
        'source':'https://docs.google.com/spreadsheets/d/1ARkIvh0D-gltZeiwniUonMNrm0Mi1s2meZ9FUjutXOE/',
        'destination':{'field':{'name':'recipe_name','suffix':' DV Targeter','kind':'string','order':3,'default':'','description':'Name of Google Sheet to create.'}}
      }
    }
  },
  {
    'dv_targeter':{
      '__comment':'Depending on users choice, execute a different part of the solution.',
      'auth_dv':{'field':{'name':'auth_dv','kind':'authentication','order':1,'default':'user','description':'Credentials used for dv.'}},
      'auth_sheets':{'field':{'name':'auth_sheet','kind':'authentication','order':2,'default':'user','description':'Credentials used for sheet.'}},
      'auth_bigquery':{'field':{'name':'auth_bigquery','kind':'authentication','order':3,'default':'service','description':'Credentials used for bigquery.'}},
      'sheet':{'field':{'name':'recipe_name','suffix':' DV Targeter','kind':'string','order':4,'default':'','description':'Name of Google Sheet to create.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','order':5,'default':'','description':'Name of Google BigQuery dataset to create.'}},
      'command':{'field':{'name':'command','kind':'choice','choices':['Clear','Load','Preview','Update'],'order':6,'default':'Load','description':'Action to take.'}},
      'first_and_third':{'field':{'name':'first_and_third','kind':'boolean','order':6,'default':False,'description':'Load first and third party data (may be slow). If not selected, enter audience identifiers into sheet manually.'}}
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(CONFIG, TASKS, force=True)
